In [ ]:
# 一个简单的例子
import torch
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = torch.tensor([10.0, 20.0, 30.0], requires_grad=True)
xx = x * x
yy = y * y
xy = x * y
xx.retain_grad()  # 保留非叶子张量的梯度
yy.retain_grad()
xy.retain_grad()
z = 10 * (xx + yy + 2 * xy)
z.retain_grad()
s = z.sum()
s.backward()
print('x.grad:\n ', x.grad)
print('y.grad:\n ', y.grad)
print('xx.grad:\n ', xx.grad)
print('yy.grad:\n ', yy.grad)
print('xy.grad:\n ', xy.grad)
print('z.grad:\n ', z.grad)  # tensor([1., 1., 1.])

In [ ]:
# 也可以直接对 `z` 调用 `backward`，这将会触发从 `z` 开始的整个计算图的反向传播。
# 不过需要注意的是，在计算图中，只有当计算结果是一个标量（单个数值）时调用 `backward()` 最为常用。
# 因为如果张量包含多个元素时，在调用 `backward` 时必须明确指定它要传播的梯度（通常被称为 `gradient` 参数）。
# 在上面的例子中，`z` 是一个包含多个元素的张量，因此在调用 `z.backward()` 时，需要指定 `gradient` 参数。
import torch
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = torch.tensor([10.0, 20.0, 30.0], requires_grad=True)
xx = x * x
yy = y * y
xy = x * y
xx.retain_grad()  # 保留非叶子张量的梯度
yy.retain_grad()
xy.retain_grad()
z = 10 * (xx + yy + 2 * xy)
z.retain_grad()
z.backward(torch.ones_like(z))  # 直接对 z 调用 backward，并指定 gradient 参数。此处使用 torch.ones_like(z) 等效于前一例子中使用 s = z.sum()
                                # 计算 s.backward() 的结果
print('x.grad:\n ', x.grad)
print('y.grad:\n ', y.grad)
print('xx.grad:\n ', xx.grad)
print('yy.grad:\n ', yy.grad)
print('xy.grad:\n ', xy.grad)
print('z.grad:\n ', z.grad)  # tensor([1., 1., 1.])

In [ ]:
# 对于非 s = z.sum() 的情况
import torch
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = torch.tensor([10.0, 20.0, 30.0], requires_grad=True)
xx = x * x
yy = y * y
xy = x * y
xx.retain_grad()  # 保留非叶子张量的梯度
yy.retain_grad()
xy.retain_grad()
z = 10 * (xx + yy + 2 * xy)
z.retain_grad()
s = z.sum() * 0.3  # 为了区别于第一个例子，我们使 s = z.sum() * 0.3
s.backward()
print('x.grad:\n ', x.grad)
print('y.grad:\n ', y.grad)
print('xx.grad:\n ', xx.grad)
print('yy.grad:\n ', yy.grad)
print('xy.grad:\n ', xy.grad)
print('z.grad:\n ', z.grad)  # tensor([0.3., 0.3., 0.3.])

In [ ]:
# 对于 s = z.sum() * 0.3, 如果直接从 z 计算梯度，则需要将 z.backward 函数的参数 gradient 乘 0.3
import torch
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = torch.tensor([10.0, 20.0, 30.0], requires_grad=True)
xx = x * x
yy = y * y
xy = x * y
xx.retain_grad()  # 保留非叶子张量的梯度
yy.retain_grad()
xy.retain_grad()
z = 10 * (xx + yy + 2 * xy)
z.retain_grad()
z.backward(torch.ones_like(z) * 0.3)  # 直接对 z 调用 backward，并指定 gradient 参数。此处使用 torch.ones_like(z) * 0.3 等效于前一例子中使用 s = z.sum() * 0.3
                                # 计算 s.backward() 的结果
print('x.grad:\n ', x.grad)
print('y.grad:\n ', y.grad)
print('xx.grad:\n ', xx.grad)
print('yy.grad:\n ', yy.grad)
print('xy.grad:\n ', xy.grad)
print('z.grad:\n ', z.grad)  # tensor([0.3., 0.3., 0.3.])

In [ ]:
# 使用 torch.autograd.Function 复刻最初的例子
import torch

class CustomFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, y):
        # 计算前向结果，并保存必要的上下文信息，供后向传播时使用
        xx = x * x
        yy = y * y
        xy = x * y
        z = 10 * (xx + yy + 2 * xy)
        
        # 保存中间结果
        ctx.save_for_backward(x, y)
        
        return z

    @staticmethod
    def backward(ctx, grad_output):
        # 从上下文中恢复前向传播时保存的变量
        x, y = ctx.saved_tensors
        
        # 计算每个中间变量的梯度
        grad_x = grad_output * (20 * x + 20 * y)
        grad_y = grad_output * (20 * y + 20 * x)
        
        # 返回与输入张量数量相同的梯度
        return grad_x, grad_y

# 测试自定义的 autograd.Function
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = torch.tensor([10.0, 20.0, 30.0], requires_grad=True)

# 使用自定义函数进行前向传播
z = CustomFunction.apply(x, y)

# 调用backward方法，计算梯度
z.sum().backward()

# 打印梯度结果
print('x.grad:\n ', x.grad)
print('y.grad:\n ', y.grad)

In [ ]:
# 自定义的 autograd.Function 的 forward 方法可以输出多个张量。
# 若 forward 输出了多个张量，backward 也需要增加参数，位于 ctx 这个参数后的参数与 forward 方法输出的多个张量的梯度一一对应。
# 这个例子中虽然 backward 函数接受了多个梯度参数，但除了第一个梯度参数外，其余的都没有被使用

# 扩展前向函数的输出以包括中间变量
class ExtendedCustomFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, y):
        xx = x * x
        yy = y * y
        xy = x * y
        z = 10 * (xx + yy + 2 * xy)
        
        # 保存上下文
        ctx.save_for_backward(x, y)
        
        return z, xx, yy, xy  # 除了输出 z 外，同时输出 xx yy xy，用于打印

    @staticmethod
    def backward(ctx, grad_z, grad_xx, grad_yy, grad_xy):  # grad_z, grad_xx, grad_yy, grad_xy 对应于 forward 的输出张量。
                                                           # 由于我们只计算 dz/dx 与 dz/dy，所以下面的计算中只用到了 grad_z
        x, y = ctx.saved_tensors
        
        grad_x = grad_z * (20 * x + 20 * y)
        grad_y = grad_z * (20 * y + 20 * x)
        
        # 返回梯度，确保每个变量的梯度都计算并返回
        return grad_x, grad_y

# 测试自定义的 autograd.Function
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = torch.tensor([10.0, 20.0, 30.0], requires_grad=True)

# 使用自定义函数进行前向传播
z, xx, yy, xy = ExtendedCustomFunction.apply(x, y)
print('z:\n ', z)
print('xx:\n ', xx)
print('yy:\n ', yy)
print('xy:\n ', xy)

# 调用 backward 方法，计算梯度
z.sum().backward()

# 打印梯度结果
print('x.grad:\n', x.grad)
print('y.grad:\n', y.grad)

In [ ]:
# 自定义的 autograd.Function 的 backward 方法接受多个梯度参数的另一个例子。
# 当 forward 方法返回多个张量时，每个张量在反向传播时都需要计算梯度，backward 方法接受的多个梯度参数就是对这些张量的梯度。

import torch

class MultipleOutputsFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        y1 = x * x
        y2 = x * 3
        ctx.save_for_backward(x)
        return y1, y2

    @staticmethod
    def backward(ctx, grad_y1, grad_y2):
        x, = ctx.saved_tensors
        grad_x = grad_y1 * 2 * x + grad_y2 * 3
        return grad_x

x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y1, y2 = MultipleOutputsFunction.apply(x)
z = y1 + 10 * y2
z.sum().backward()
print('x.grad:\n ', x.grad)

In [ ]:
# 自定义 torch.autograd.Function，并在 forward 方法中加入非张量类型入参用于设置 forward 与 backward 函数的行为。
# 虽然这些非张量参数在自动微分过程中不会被追踪和求导，但它们可以通过上下文传递在反向传播中使用。

import torch
from typing import NamedTuple

class CustomSettings(NamedTuple):
    print_debug_info: bool

class CustomFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, y, custom_settings):  # 非张量类型参数 custem_settings 用于控制 forward 和 backward 函数的行为
        # 计算前向结果，并保存必要的上下文信息，供后向传播时使用
        xx = x * x
        yy = y * y
        xy = x * y
        z = 10 * (xx + yy + 2 * xy)
        
        if custom_settings.print_debug_info:
            print(f"debug info: xx={xx}, yy={yy}, xy={xy}")
        
        # 保存中间结果
        ctx.save_for_backward(x, y)
        ctx.custom_settings = custom_settings
        
        return z

    @staticmethod
    def backward(ctx, grad_output):
        # 从上下文中恢复前向传播时保存的变量
        x, y = ctx.saved_tensors
        custom_settings = ctx.custom_settings
        
        if custom_settings.print_debug_info:
            print(f"debug info: grad_output={grad_output}")
        
        # 计算每个中间变量的梯度
        grad_x = grad_output * (20 * x + 20 * y)
        grad_y = grad_output * (20 * y + 20 * x)
        
        # custem_settings 位置返回 None
        return grad_x, grad_y, None

# 测试自定义的 autograd.Function
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = torch.tensor([10.0, 20.0, 30.0], requires_grad=True)

custem_settings = CustomSettings(
    print_debug_info = True,
)

# 使用自定义函数进行前向传播
z = CustomFunction.apply(x, y, custem_settings)

# 调用backward方法，计算梯度
z.sum().backward()

# 打印梯度结果
print('x.grad:\n ', x.grad)
print('y.grad:\n ', y.grad)